# December 2017: Advent of Code
## Parts 7-12

## Common imports & library functions

In [1]:
from collections import defaultdict, namedtuple
import doctest
import heapq
import itertools
import math
import numpy as np
import re

# Cribbed from norvig@
def nth(iterable, n, default=None):
    "Returns the nth item of iterable, or a default value"
    return next(itertools.islice(iterable, n, None), default)

def np_impulse(shape, idx, value, dtype=np.int):
    data = np.zeros(shape, dtype=dtype)
    data[idx] = value
    return data

## Day 7: Recursive Circus

In [ ]:
node_re = re.compile(r"([a-z]+) \(([\-0-9]+)\)(?: -> ([\w, ]+))?")

Node = namedtuple('Node', ['id', 'weight', 'children'])
Tree = namedtuple('Tree', ['root', 'nodes'])

def parse_node(text):
    """
    >>> parse_node("ktlj (57)")
    Node(id='ktlj', weight=57, children=[])
    >>> parse_node("xyz (-7) -> abc, def, ghi")
    Node(id='xyz', weight=-7, children=['abc', 'def', 'ghi'])
    """
    node_id, weight, children = node_re.match(text).groups()
    weight = int(weight)
    children = [c.strip() for c in children.split(',')] if children else []
    return Node(node_id, weight, children)

def parse_tree(text):
    """
    >>> tree = parse_tree("ktlj (57)\\nxyz (-7) -> ktlj")
    >>> tree.root
    'xyz'
    """
    nodes = {}
    for line in text.splitlines():
        line = line.strip()
        if not line: continue
        node = parse_node(line)
        nodes[node.id] = node
    root = get_root(nodes)
    return Tree(root, nodes)


def tree_weight(tree, node_id=None):
    """
    >>> t = Tree('xyz', {'xyz': Node('xyz', 10, ['abc']), 'abc': Node('abc', -2, [])})
    >>> tree_weight(t, 'xyz')
    8
    >>> tree_weight(t, 'abc')
    -2
    >>> tree_weight(t)
    8
    """
    node_id = node_id or tree.root
    node = tree.nodes[node_id]
    return node.weight + sum(tree_weight(tree, c) for c in node.children)

def verify_tree(tree, node_id=None):
    node_id = node_id or tree.root
    node = tree.nodes[node_id]
    if not node.children:
        return node.weight
    ws = [verify_tree(tree, c) for c in node.children]
    if len(set(ws)) != 1:
        bad_idx = list((ws[i] == ws[i-1]) or (ws[i] == ws[i-2]) 
                       for i in range(len(ws))).index(False)
        bad_node = tree.nodes[node.children[bad_idx]]
        bad_weight = ws[bad_idx]
        good_weight = ws[bad_idx - 1]
        correction = (good_weight - bad_weight)
        raise Exception('Failed verification at node <{}> due to child <{}>'.format(node_id, 
                                                                                    bad_node.id),
                        node.children[bad_idx],
                        bad_node.weight + correction)
    else:
        return node.weight + sum(ws)
        

def get_root(nodes):
    """
    >>> get_root({'xyz': Node('xyz', 1, children=['abc']), 'abc': Node('abc', 2, [])})
    'xyz'
    """
    all_nodes = set(nodes)
    child_nodes = set(
        itertools.chain.from_iterable(n.children for n in nodes.values()))
    return (all_nodes - child_nodes).pop()
        
def solve_bottom_program(tree):
    return tree.root

In [ ]:
doctest.testmod(verbose=True)

test_tree = parse_tree("""
pbga (66)
xhth (57)
ebii (61)
havc (66)
ktlj (57)
fwft (72) -> ktlj, cntj, xhth
qoyq (66)
padx (45) -> pbga, havc, qoyq
tknk (41) -> ugml, padx, fwft
jptl (61)
ugml (68) -> gyxo, ebii, jptl
gyxo (61)
cntj (57)
""")

print('Root is:', test_tree.root)
assert test_tree.root == 'tknk'

try:
    verify_tree(test_tree)
except Exception as e:
    msg, bad_node, corrected_weight = e.args
    assert corrected_weight == 60

In [ ]:
# Final answer
with open('day7.txt') as f:
    tree = parse_tree(f.read())
    print('Part 1: root node is', tree.root)
    try:
        verify_tree(tree)
    except Exception as e:
        print('Part 2: corrected weight is', e.args[2])

## Day 8: I Heard You Like Registers

In [2]:
def Registers(init_values={}):
    return defaultdict(int, init_values)

def print_registers(registers):
    return ' | '.join('{}: {}'.format(*kv) for kv in sorted(registers.items()))
    
def parse_instruction(line):
    """
    >>> parse_instruction('a inc 5 if b == 3')
    ('a', 'inc', 5, 'b', '==', 3)
    """
    var1, op, opval, _, var2, comp, compval = line.split()
    return (var1, op, int(opval), 
            var2, comp, int(compval))

def parse_program(program_text):
    """
    >>> parse_program('a inc 5 if b == 3')
    [('a', 'inc', 5, 'b', '==', 3)]
    >>> parse_program('a inc 5 if b == 3\\nc dec -10 if a >= 1')
    [('a', 'inc', 5, 'b', '==', 3), ('c', 'dec', -10, 'a', '>=', 1)]
    """
    return [parse_instruction(line) for line in program_text.splitlines() if line]

def eval_cond(cond, registers):
    """
    >>> eval_cond(('a', '==', 4), {'a': 4})
    True
    >>> eval_cond(('b', '>', 4), {'b': -3})
    False
    """
    reg, op, val = cond
    return eval('registers["{}"] {} {}'.format(reg, op, val))

def eval_op(op, registers):
    reg, op, val = op
    op = '+=' if op == 'inc' else '-='
    expr = 'registers["{}"] {} {}'.format(reg, op, val)
    eval(compile(expr, '<string>', 'single'))

def run_instruction(instruction, registers):
    """
    >>> registers = Registers({'b': 3})
    >>> print_registers(run_instruction(('a', 'inc', 5, 'b', '==', '3'), registers))
    'a: 5 | b: 3'
    """
    if eval_cond(instruction[3:], registers):
        eval_op(instruction[:3], registers)
    return registers

def run_program(instructions, registers=None):
    """
    >>> registers = Registers({'b': 3})
    >>> print_registers(run_program([('a', 'inc', 5, 'b', '==', '3')], registers))
    'a: 5 | b: 3'
    """
    registers = registers or Registers()
    for instruction in instructions:
        run_instruction(instruction, registers)
    return registers

def solve_max_register_value(program_text):
    instructions = parse_program(program_text)
    registers = Registers()
    historical_max = 0
    for instruction in instructions:
        run_instruction(instruction, registers)
        historical_max = max(historical_max, max(registers.values()))
    return max(registers.values()), historical_max

In [3]:
doctest.testmod(verbose=True)

test_program = """
b inc 5 if a > 1
a inc 1 if b < 5
c dec -10 if a >= 1
c inc -20 if c == 10 
"""

current_max, historical_max = solve_max_register_value(test_program)
assert current_max == 1
assert historical_max == 10

Trying:
    eval_cond(('a', '==', 4), {'a': 4})
Expecting:
    True
ok
Trying:
    eval_cond(('b', '>', 4), {'b': -3})
Expecting:
    False
ok
Trying:
    parse_instruction('a inc 5 if b == 3')
Expecting:
    ('a', 'inc', 5, 'b', '==', 3)
ok
Trying:
    parse_program('a inc 5 if b == 3')
Expecting:
    [('a', 'inc', 5, 'b', '==', 3)]
ok
Trying:
    parse_program('a inc 5 if b == 3\nc dec -10 if a >= 1')
Expecting:
    [('a', 'inc', 5, 'b', '==', 3), ('c', 'dec', -10, 'a', '>=', 1)]
ok
Trying:
    registers = Registers({'b': 3})
Expecting nothing
ok
Trying:
    print_registers(run_instruction(('a', 'inc', 5, 'b', '==', '3'), registers))
Expecting:
    'a: 5 | b: 3'
ok
Trying:
    registers = Registers({'b': 3})
Expecting nothing
ok
Trying:
    print_registers(run_program([('a', 'inc', 5, 'b', '==', '3')], registers))
Expecting:
    'a: 5 | b: 3'
ok
7 items had no tests:
    __main__
    __main__.Registers
    __main__.eval_op
    __main__.np_impulse
    __main__.nth
    __main__.print_r

In [4]:
# Final answer
with open('day8.txt') as f:
    current_max, historical_max = solve_max_register_value(f.read())
    print('Part 1: current max value is', current_max)
    print('Part 1: historical max value is', historical_max)

Part 1: current max value is 4647
Part 1: historical max value is 5590
